In [1]:
##
#Yin Ma
#6 18 2020, based on 0417 01logit, update biogeme version,start with simple logit w/ asc, but without scales
# biogeme 3.2.6 became available 06 05 2020, update it by:pip install biogeme --upgrade

#logit, panel data
 

In [2]:
# check biogem version 
import biogeme.version as ver
print(ver.getText())

biogeme 3.2.6 [2020-06-18]
Version entirely written in Python
Home page: http://biogeme.epfl.ch
Submit questions to https://groups.google.com/d/forum/biogeme
Michel Bierlaire, Transport and Mobility Laboratory, Ecole Polytechnique Fédérale de Lausanne (EPFL)



In [3]:
import pandas as pd
import numpy as np

import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable, bioDraws, PanelLikelihoodTrajectory, MonteCarlo, log
import biogeme.expressions as ex

In [4]:
# Read the data
# 04 27 introduce scale variables (the ones are significant in essay 2)
df = pd.read_csv("agree0623_all_0427.csv")

#df.head(10)
#check missing values
#df.isnull().sum()

In [5]:
database = db.Database("agree0623_all_0427",df)

In [6]:
# They are organized as panel data. The variable ID identifies each individual.
database.panel("csid")

In [7]:
# use the names of the variable as Python variable.
globals().update(database.variables)

In [8]:
#we already remove missing values, so we don't need to exclude any observations here 
#note that weights were created before select variables and remove missing values

In [9]:
# using block specific data
#exclude=(block1==0)
#exclude=(block1==1)

#database.remove(exclude)
#next step, change weight accordingly

In [10]:
# Parameters to be estimated
#NZone variables are already scaled: zone/100
#use lg estimates as starting values 

#start with a simple model w/o scales 
B_ALTA = ex.Beta('B_ALTA',0,None,None,0)
B_ALTB = ex.Beta('B_ALTB',0,None,None,0)

B_NZONE1 = ex.Beta('B_NZONE1',-0.0682,None,None,0)
B_NZONE2 = ex.Beta('B_NZONE2',-0.0372,None,None,0)
B_NZONE3 = ex.Beta('B_NZONE3',-0.0283,None,None,0)
B_TOWNDOWN = ex.Beta('B_TOWNDOWN',0.0274,None,None,0)
B_TOWNMID = ex.Beta('B_TOWNMID',0.11435,None,None,0)
B_TOWNUP = ex.Beta('B_TOWNUP',0.205,None,None,0)
B_ECO = ex.Beta('B_ECO',0.053,None,None,0)
B_REC = ex.Beta('B_REC',0.0089,None,None,0)
B_DRY = ex.Beta('B_DRY',-0.2146,None,None,0)
B_TAX = ex.Beta('B_TAX',-0.0257,None,None,0)

## covariates, starting values from essay 2 scaled mnl
B_ASCEND = ex.Beta('B_ASCEND',0.2997,None,None,0)
B_DESCEND = ex.Beta('B_DESCEND',0.2851,None,None,0)
B_INCPP50K = ex.Beta('B_INCPP50K',0.317,None,None,0)
B_INCPP75K = ex.Beta('B_INCPP75K',0.5522,None,None,0)
B_INCPP75KM = ex.Beta('B_INCPP75KM',0.1951,None,None,0)
B_F1FLOW = ex.Beta('B_F1FLOW',-0.6279,None,None,0)
B_F3PROTUP = ex.Beta('B_F3PROTUP',0.0066,None,None,0)
B_F4CTH2O = ex.Beta('B_F4CTH2O',-0.5495,None,None,0)
B_TAXGRT = ex.Beta('B_TAXGRT',0.3198,None,None,0)

 Definition of new variables: in simulation, do not use the DefineVariable operator, as it hides the functional relationships. In particular, derivatives cannot be calculated.
 in this study, we prescale variables in the dataset and don't need to define variales here

In [11]:
# Definition of the utility functions
#start with a simple model w/o scales 

VSQ =B_NZONE1 * nzone1_sq + B_NZONE2 * nzone2_sq + B_NZONE3 * nzone3_sq + B_TOWNDOWN * towndown_sq + B_TOWNMID * townmid_sq + B_TOWNUP * townup_sq + B_ECO * eco_sq + B_REC * rec_sq + B_DRY * dry_sq + B_TAX * tax_sq
VA = B_ALTA+ B_NZONE1 * nzone1_a + B_NZONE2 * nzone2_a + B_NZONE3 * nzone3_a + B_TOWNDOWN * towndown_a + B_TOWNMID * townmid_a + B_TOWNUP * townup_a + B_ECO * eco_a + B_REC * rec_a + B_DRY * dry_a + B_TAX * tax_a
VB = B_ALTB+ B_NZONE1 * nzone1_b + B_NZONE2 * nzone2_b + B_NZONE3 * nzone3_b + B_TOWNDOWN * towndown_b + B_TOWNMID * townmid_b + B_TOWNUP * townup_b + B_ECO * eco_b + B_REC * rec_b + B_DRY * dry_b + B_TAX * tax_b

# Definition of the utility functions
scale = B_ASCEND*ascend+B_DESCEND*descend+B_INCPP50K*incpp_50k+B_INCPP75K*incpp_75k+B_INCPP75KM*incpp_75km+B_F1FLOW*f1_flow+B_F3PROTUP*f3_protup+B_F4CTH2O*f4_CTh2o+B_TAXGRT*taxgrt

VSQ =scale* (B_NZONE1 * nzone1_sq + B_NZONE2 * nzone2_sq + B_NZONE3 * nzone3_sq + B_TOWNDOWN * towndown_sq + B_TOWNMID * townmid_sq + B_TOWNUP * townup_sq + B_ECO * eco_sq + B_REC * rec_sq + B_DRY * dry_sq + B_TAX * tax_sq)
VA = scale *( B_NZONE1 * nzone1_a + B_NZONE2 * nzone2_a + B_NZONE3 * nzone3_a + B_TOWNDOWN * towndown_a + B_TOWNMID * townmid_a + B_TOWNUP * townup_a + B_ECO * eco_a + B_REC * rec_a + B_DRY * dry_a + B_TAX * tax_a)
VB = scale *( B_NZONE1 * nzone1_b + B_NZONE2 * nzone2_b + B_NZONE3 * nzone3_b + B_TOWNDOWN * towndown_b + B_TOWNMID * townmid_b + B_TOWNUP * townup_b + B_ECO * eco_b + B_REC * rec_b + B_DRY * dry_b + B_TAX * tax_b)

In [12]:
# Associate utility functions with the numbering of alternatives
V = {1: VSQ,
     2: VA,
     3: VB}

# Associate the availability conditions with the alternatives
# all alternatives are available for each individual
av = {1: 1,
      2: 1,
      3: 1}

In [13]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
logprob = models.loglogit(V,av,choice)

In [14]:
prob_sq = models.logit(V,av,1)
prob_a = models.logit(V,av,2)
prob_b = models.logit(V,av,3)

In [15]:
# Define level of verbosity
import biogeme.messaging as msg
logger = msg.bioMessage()
logger.setDebug()
#logger.setWarning()
#logger.setGeneral()
#logger.setDetailed()


In [16]:
# Estimate the parameters
biogeme = bio.BIOGEME(database,logprob)
results = biogeme.estimate(saveIterations=True)
biogeme.createLogFile()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)

biogeme.modelName = "01logit_0618"

results = biogeme.estimate()

print("Results=",results)

[17:55:40] < General >   Remove 31 unused variables from the database as only 32 are used.
[17:55:40] < Detailed >  It is suggested to scale the following variables.
[17:55:40] < Detailed >  Multiply rec_b by	0.01 because the largest (abs) value is	90.0
[17:55:40] < Detailed >  Multiply rec_a by	0.01 because the largest (abs) value is	90.0
[17:55:40] < Detailed >  Multiply eco_a by	0.01 because the largest (abs) value is	41.0
[17:55:40] < Detailed >  Multiply rec_sq by	0.01 because the largest (abs) value is	40.0
[17:55:40] < Detailed >  Multiply eco_b by	0.01 because the largest (abs) value is	41.0
[17:55:40] < Detailed >  Multiply csid by	0.0001 because the largest (abs) value is	7624.0
[17:55:40] < Detailed >  To remove this feature, set the parameter suggestScales to False when creating the BIOGEME object.
[17:55:44] < Detailed >  Log likelihood (N = 7414):  -7614.592
[17:55:44] < Detailed >  ** Optimization: BFGS with trust region for simple bounds
[17:55:44] < Debug >     B_ALTA: